# Assignment 7

I think it needed a redo rather than straight from hand on because we've skipped training data so it won't be accurate<br>
and it's hard to fix that in there so I reorganized the whole thing

`Assignment details: `

Develop an Optuna optimized model that accounts for both title and body text.

- Show that you can parallelize optuna process and use all cpus. (1 point)
- Fine-tune the model to achieve the highest cross-validation (CV) accuracy:
    - Final f1 > 0.80 will earn 1 point
    - Final f1 <= 0.80 will earn 0.5 point
- Showcase enhancements in data preprocessing techniques. (1 point)
- Explain how you improved hyperparameter tuning. (1 point)
- Finally, integrate the best-configured model into a Flask application. (1 point)

In [35]:
import re
import string
import logging
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from multiprocessing import Pool

# Precompile translation table (if needed)
TRANSLATION_TABLE = str.maketrans('', '', '!"#$%&\'()*+,.<=>?@[]^`{|}~' + u'\xa0')

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

def preprocess(text, stopwords_set, ps):
    logging.info(f"Processing: {text[:30]}...")  # This will now be logged
    words = text.split()
    words = [w for w in words if w.lower() not in stopwords_set]
    words = [ps.stem(w) for w in words]
    processed_text = ' '.join(words)
    logging.info(f"Processed: {processed_text[:30]}...")
    return processed_text

# Function for initializing globals in worker processes.
def initialize_pool(stopwords_set_arg, stemmer_arg):
    global stopwords_set, ps
    stopwords_set = stopwords_set_arg
    ps = stemmer_arg

# Top-level (picklable) wrapper function for multiprocessing.
def preprocess_wrapper_global(text):
    # Uses the globals that were set in initialize_pool.
    return preprocess(text, stopwords_set, ps)


In [36]:
# Configuration for limiting training size (toggle off by setting LIMIT_TRAINING = False)
LIMIT_TRAINING = True
SAMPLE_SIZE = 100 # Change to a higher number when you're ready

# Load dataset
dataset = pd.read_json('../Week 10/resource/embold_train.json')

# Label transformations
dataset.loc[dataset['label'] > 0, 'label'] = -1
dataset.loc[dataset['label'] == 0, 'label'] = 1
dataset.loc[dataset['label'] == -1, 'label'] = 0

# Optionally limit dataset size for faster debugging
if LIMIT_TRAINING:
    dataset = dataset.sample(n=SAMPLE_SIZE, random_state=42).reset_index(drop=True)

print('Dataset shape:', dataset.shape)


Dataset shape: (100, 3)


In [37]:
# Preprocess title and body without parallel processing (sequential processing)
cleaned_title = [preprocess(text, stopwords_set, ps) for text in dataset['title']]
cleaned_body = [preprocess(text, stopwords_set, ps) for text in dataset['body']]

# Combine the cleaned title and body into a single text field
data_texts = pd.DataFrame({
    'text': [t + ' ' + b for t, b in zip(cleaned_title, cleaned_body)]
})

# Labels
y = dataset['label']

print('Preprocessed texts sample:')
print(data_texts.head())


2025-03-12 16:00:42,615 - Processing: install error at task  gce-net...
2025-03-12 16:00:42,615 - Processed: instal error task gce-network ...
2025-03-12 16:00:42,615 - Processing: feature request add remote url...
2025-03-12 16:00:42,615 - Processed: featur request add remot url i...
2025-03-12 16:00:42,615 - Processing: add backface culling option...
2025-03-12 16:00:42,615 - Processed: add backfac cull option...
2025-03-12 16:00:42,615 - Processing: logo.png is overwritten in clo...
2025-03-12 16:00:42,615 - Processed: logo.png overwritten clone ani...
2025-03-12 16:00:42,615 - Processing: marble != marble  quark...
2025-03-12 16:00:42,615 - Processed: marbl != marbl quark...
2025-03-12 16:00:42,615 - Processing: change the 'people' link in th...
2025-03-12 16:00:42,615 - Processed: chang 'people' link navbar rea...
2025-03-12 16:00:42,627 - Processing: scope/cortex links enabled whe...
2025-03-12 16:00:42,628 - Processed: scope/cortex link enabl agent/...
2025-03-12 16:00:42,628 - 

Preprocessed texts sample:
                                                text
0  instal error task gce-network : creat network ...
1  featur request add remot url imag code:\r imag...
2  add backfac cull option add option enable/dis ...
3  logo.png overwritten clone anim anim preexist ...
4  marbl != marbl quark want marbl stair 12k marb...


In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Use both title and body (combined) for TF-IDF vectorization
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1), max_features=10000)
X = tfidf_vectorizer.fit_transform(data_texts['text'])

# Split data for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

print('TF-IDF matrix shape:', X.shape)

TF-IDF matrix shape: (100, 2099)


In [39]:
import lightgbm as lgb
import optuna
from sklearn.model_selection import cross_val_score

# Objective function for Optuna hyperparameter tuning
def objective(trial):
    param = {
        "objective": "binary",
        "metric": "binary_logloss",
        "boosting_type": "gbdt",
        "verbosity": -1,
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100)
    }
    
    model = lgb.LGBMClassifier(**param, n_jobs=-1, random_state=42)
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='f1_macro', n_jobs=-1).mean()
    return score

# Run the Optuna study using all CPUs
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, n_jobs=-1)

print('Best trial:')
print(f"F1 Macro: {study.best_trial.value:.4f}")
print('Params:')
for key, value in study.best_trial.params.items():
    print(f"  {key}: {value}")

[I 2025-03-12 16:00:43,024] A new study created in memory with name: no-name-161bb0a1-a943-4c75-9ccc-a63b38c4a617
[I 2025-03-12 16:00:43,213] Trial 0 finished with value: 0.33939393939393936 and parameters: {'lambda_l1': 0.0016589801331090542, 'lambda_l2': 9.79415930939998e-06, 'num_leaves': 154, 'feature_fraction': 0.9639437989453765, 'bagging_fraction': 0.6795414251607695, 'bagging_freq': 2, 'min_child_samples': 74}. Best is trial 0 with value: 0.33939393939393936.
[I 2025-03-12 16:00:43,244] Trial 1 finished with value: 0.33939393939393936 and parameters: {'lambda_l1': 0.007463823473575728, 'lambda_l2': 0.00011839401360827678, 'num_leaves': 17, 'feature_fraction': 0.9942783014325637, 'bagging_fraction': 0.6217130574678711, 'bagging_freq': 6, 'min_child_samples': 34}. Best is trial 0 with value: 0.33939393939393936.
[I 2025-03-12 16:00:43,303] Trial 3 finished with value: 0.33939393939393936 and parameters: {'lambda_l1': 0.0015510582838889246, 'lambda_l2': 0.0001034718102190144, 'num

Best trial:
F1 Macro: 0.6500
Params:
  lambda_l1: 3.7099138908033166e-05
  lambda_l2: 2.9688092004540296e-05
  num_leaves: 3
  feature_fraction: 0.9292435895396467
  bagging_fraction: 0.6615864872343277
  bagging_freq: 5
  min_child_samples: 6


In [40]:
# Train final model with best hyperparameters
best_params = study.best_trial.params
final_model = lgb.LGBMClassifier(**best_params, n_jobs=-1, random_state=42)
final_model.fit(X_train, y_train)

from sklearn.metrics import f1_score
y_pred = final_model.predict(X_test)
final_f1 = f1_score(y_test, y_pred, average='macro')
print(f"Final Test F1 Macro Score: {final_f1:.4f}")

[LightGBM] [Warning] feature_fraction is set=0.9292435895396467, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9292435895396467
[LightGBM] [Warning] lambda_l1 is set=3.7099138908033166e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.7099138908033166e-05
[LightGBM] [Warning] lambda_l2 is set=2.9688092004540296e-05, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.9688092004540296e-05
[LightGBM] [Warning] bagging_fraction is set=0.6615864872343277, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6615864872343277
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.9292435895396467, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9292435895396467
[LightGBM] [Warning] lambda_l1 is set=3.7099138908033166e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.7099138908033166e-05
[LightGBM] [W

In [42]:
import pickle

resource_path = '../Assignment 7/resource'

with open(f'{resource_path}/github_bug_prediction_tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)

with open(f'{resource_path}/github_bug_prediction_final_model.pkl', 'wb') as f:
    pickle.dump(final_model, f)

print('Components saved to', resource_path)

Components saved to ../Assignment 7/resource


## Flask Application

The following Flask app loads the saved vectorizer and model, preprocesses the incoming `title` and `body` (concatenated), and returns a prediction.

In [ ]:
from flask import Flask, request, jsonify
from scipy.sparse import hstack
import pickle

app = Flask(__name__)

# Load components from '../Assignment 7/resource'
resource_path = '../Assignment 7/resource'
with open(f'{resource_path}/github_bug_prediction_tfidf_vectorizer.pkl', 'rb') as f:
    app.tfidf_vectorizer = pickle.load(f)
with open(f'{resource_path}/github_bug_prediction_final_model.pkl', 'rb') as f:
    app.model = pickle.load(f)

# Reuse the same preprocess function
def flask_preprocess(text):
    return preprocess(text, set(stopwords.words('english')), PorterStemmer())

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    if not data or ('title' not in data and 'body' not in data):
        return jsonify({'status': 'error', 'message': 'Missing title or body'}), 400
    
    title = data.get('title', '')
    body = data.get('body', '')
    raw_text = (title + ' ' + body).strip()
    
    processed_text = flask_preprocess(raw_text)
    X_input = app.tfidf_vectorizer.transform([processed_text])
    prob = app.model.predict_proba(X_input)[0][1]  # assuming positive class at index 1
    
    result = {
        'status': 'success',
        'predict_as': 'bug' if prob > 0.5 else 'not bug',
        'bug_prob': prob
    }
    return jsonify(result)


In [46]:
app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


2025-03-12 16:05:45,287 - WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
2025-03-12 16:05:45,288 - Press CTRL+C to quit
